# testing the dimension reduction of word2vec models

In [27]:
import fasttext
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import gensim
from pprint import pprint

In [ ]:
model_filename = '/mnt/data/data/models-fasttext/cc.sk.300-lemma_500k_min2_excluded.vec.bin'
model_big = KeyedVectors.load_word2vec_format(model_filename, binary=True)
model_big.save_word2vec_format(model_filename+'.txt', binary=False)
#model = fasttext.load_model(model_filename+'.txt')

In [7]:
vec = model_big.vectors

In [12]:
print(len(vec), len(vec[0]))

148816 300


In [20]:
mapping_size = 100000
dim = 100
#X = vec[:mapping_size]
X = vec
X = X - X.mean(axis=0, dtype=np.float32)
C = np.divide(np.matmul(X.T, X), X.shape[0] - 1, dtype=np.float32)
_, U = np.linalg.eig(C)
eigv = U[:, :dim]
print(len(eigv), len(eigv[0]))
#X_reduced = np.matmul(X_orig, eigv)

300 100


In [22]:
X_reduced = np.matmul(vec, eigv)
print(len(X_reduced), len(X_reduced[0]))

148816 100


In [25]:
new_vector_space = gensim.models.keyedvectors.Word2VecKeyedVectors(100)

new_vector_space.add(model_big.index2word,X_reduced)

In [51]:
model_filename_reduced = '/mnt/data/data/models-fasttext/cc.sk.100-lemma_500k_min2_excluded.vec.bin'
new_vector_space.save_word2vec_format(model_filename_reduced, binary=True)

In [28]:
pprint(model_big.most_similar('otec'))
pprint(model_big.most_similar_cosmul(positive=['žena', 'kráľ'], negative=['muž']))

[('syn', 0.7990518808364868),
 ('otcov', 0.7085798978805542),
 ('matka', 0.7008340358734131),
 ('syna', 0.6916787624359131),
 ('brat', 0.6684491634368896),
 ('mama', 0.6567193269729614),
 ('otcova', 0.6508373022079468),
 ('ded', 0.6416566371917725),
 ('rodič', 0.6386483907699585),
 ('otcovstvo', 0.6213870048522949)]
[('kráľovná', 0.8815598487854004),
 ('kráľovna', 0.8545932769775391),
 ('kráľov', 0.8528066277503967),
 ('kráľovstvo', 0.8361407518386841),
 ('kráľovena', 0.826107919216156),
 ('panovník', 0.8226426839828491),
 ('kráľový', 0.8216139078140259),
 ('kráľovič', 0.8085234761238098),
 ('král', 0.8070907592773438),
 ('panovníčka', 0.8058594465255737)]


In [29]:
pprint(new_vector_space.most_similar('otec'))
pprint(new_vector_space.most_similar_cosmul(positive=['žena', 'kráľ'], negative=['muž']))

[('syn', 0.8788043260574341),
 ('syna', 0.7982639074325562),
 ('brat', 0.7961896657943726),
 ('matka', 0.7773278951644897),
 ('ježiš', 0.7616710662841797),
 ('boh', 0.7604951858520508),
 ('otcov', 0.7484495639801025),
 ('učeník', 0.7449938058853149),
 ('ježiša', 0.7438328266143799),
 ('boží', 0.7251167893409729)]
[('kráľovná', 0.9018763899803162),
 ('panovníčka', 0.8892630338668823),
 ('panovník', 0.876750648021698),
 ('kráľovena', 0.8761286735534668),
 ('kráľovna', 0.8754265904426575),
 ('kráľov', 0.8744796514511108),
 ('kráľovstvo', 0.8699237108230591),
 ('trón', 0.8657398819923401),
 ('princ', 0.8638396859169006),
 ('panovníček', 0.8636679649353027)]


In [33]:
import time
start = time.time()
for i in range(1000):
    new_vector_space.most_similar_cosmul(positive=['žena', 'kráľ'], negative=['muž'])

print("\ntime: ",time.time()-start)


time:  26.869284629821777


In [34]:
import time
start = time.time()
for i in range(1000):
    model_big.most_similar_cosmul(positive=['žena', 'kráľ'], negative=['muž'])

print("\ntime: ",time.time()-start)


time:  56.53255081176758


In [56]:
%%prun
# conclusion: dimension does not affect the speed much 
# (50000 wmd comparison of 3-4words dim300:12s vs dim100: 11s)

list1 = ['bezpečnostný', 'opatrenie', 'zviera']
list2 = ['tajný', 'služba', 'auto', 'dom']

l1 = "_".join(list1)
l2 = "_".join(list2)

#print(model_big.wmdistance(l1.split('_'), l2.split('_')))
print(model_big.wmdistance(list1, list2))
start = time.time()
for i in range(50000):
    #model_big.wmdistance(list1, list2)
    model_big.wmdistance(l1.split('_'), l2.split('_'))

print("\ntime: ",time.time()-start)  

#print(new_vector_space.wmdistance(l1.split('_'), l2.split('_')))
print(new_vector_space.wmdistance(list1, list2))
start = time.time()
for i in range(50000):
    #new_vector_space.wmdistance(list1, list2)
    new_vector_space.wmdistance(l1.split('_'), l2.split('_'))

print("\ntime: ",time.time()-start)  

0.516289522139179

time:  18.12471628189087
0.41870491548955485

time:  17.27615451812744
 

In [54]:
#%%timeit

#print(model_big.wmdistance(l1.split('_'), l2.split('_')))
print(model_big.n_similarity(list1, list2))
start = time.time()
for i in range(50000):
    #model_big.wmdistance(list1, list2)
    model_big.n_similarity(l1.split('_'), l2.split('_'))

print("\ntime: ",time.time()-start)  

#print(new_vector_space.wmdistance(l1.split('_'), l2.split('_')))
print(new_vector_space.n_similarity(list1, list2))
start = time.time()
for i in range(50000):
    #new_vector_space.wmdistance(list1, list2)
    new_vector_space.n_similarity(l1.split('_'), l2.split('_'))

print("\ntime: ",time.time()-start)  

0.6008658

time:  2.108680486679077
0.6845576186124238

time:  1.86521577835083
0.6008658

time:  2.062317132949829
0.6845576186124238

time:  1.8391196727752686
0.6008658

time:  2.074707508087158
0.6845576186124238

time:  1.8309576511383057
0.6008658

time:  2.0736265182495117
0.6845576186124238

time:  1.8727285861968994
0.6008658

time:  2.120185375213623
0.6845576186124238

time:  1.8817689418792725
0.6008658

time:  2.09938383102417
0.6845576186124238

time:  1.8597073554992676
0.6008658

time:  2.0889620780944824
0.6845576186124238

time:  1.9067225456237793
0.6008658

time:  2.1094563007354736
0.6845576186124238

time:  1.882667064666748
3.96 s ± 38.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
